# Imports
Import python libraries

In [433]:
import os

# Set dataset parameters
Choose the attributes to load the dataset.

In [434]:


detectors = ['mtcnn_serfiq']  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_models = ['Facenet512','QMagFace','ArcFace']  # 'QMagFace', "ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'
quality_models = ['ser_fiq','tface'] #  'ser_fiq','tface'


# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [435]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'

calibration_db0 = ['XQLFW','SCFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',
image_filters0 = ['yaw', 'gender', 'race','emotion']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'

calibration_db = ['XQLFW'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW','UTKface','ENFSI',
#'XQLFW',"SCFace",'ForenFace','LFW',

image_filters = []   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'
#'yaw', 'gender', 'race','emotion'

face_image_filters = []  # 'confusion_score'
quality_filters = [] # 'quality_group'

#all the filters in one dictionary
filters = {'image': image_filters, 'face_image': face_image_filters,'quality_image': quality_filters}

n_calibration_pairs = 10000



## Validation

In [436]:
enfsi_years = [2015]

# Video parameters

In [437]:
import numpy as np
#quality_dropout = [0.01, 0.02, 0.05, 0.10, 0.20, 0.5, 1]
quality_dropout = list(np.linspace(0.1,1.0,10))

## Other parameters
Rest of the configuration

In [438]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [439]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [440]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset_21'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_6')

# Preprocess data
Connect to sql database.

In [441]:
from sql_face.alchemy import get_session
from sql_face.tables import *

In [442]:
session = get_session(input_dir, db_name)

# Experiments

All parameters are specified in ExperimentalSetup, that creates a different experiment with all the posible combinations of detectors, embedding_models, calibrators, etc.

In [443]:
from lr_video_face.experiments import ExperimentalSetup

In [444]:
experimental_setup = ExperimentalSetup(detectors, 
                                        embedding_models, 
                                        quality_models,
                                        calibrators, 
                                        calibration_db, 
                                        enfsi_years,
                                        filters,
                                        metrics, 
                                        n_calibration_pairs,
                                        embedding_model_as_scorer,
                                        output_dir,
                                        session,
                                        quality_dropout)

# Results

In [445]:
'''for exp in experimental_setup:
    exp.perform()
    break  '''

'for exp in experimental_setup:\n    exp.perform()\n    break  '

In [446]:
from lr_video_face.evaluators import GlobalEvaluator  

In [447]:
global_results = GlobalEvaluator(experimental_setup)    

100%|██████████| 6/6 [02:45<00:00, 27.62s/it]


## Individual plots

In [448]:
global_results.make_experiment_plots()

## General plot

In [449]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [450]:
global_results.make_global_plot()

0.756033893290136 [0.28951688 0.5145045  0.95496388]
0.8819349774672205 [0.39890569 0.6893949  1.12547969]
0.8229548895941351 [0.34150088 0.62275689 1.03805473]
0.9784751287075095 [0.47074885 0.80615878 1.29463957]
0.9345901595904342 [0.44340379 0.76187075 1.21822699]
1.0223687329176823 [0.48528871 0.84950933 1.39554036]
1.072668145027673 [0.49410052 0.88394555 1.51827296]
1.1127319002223754 [0.50265675 0.92570793 1.63561285]
1.1435765864997236 [0.51254749 0.96673878 1.71048166]
1.17353978082501 [0.52030499 1.01314655 1.79860085]
0.9503294487600575 [0.38110658 0.67813815 1.28408444]
1.0131625057802465 [0.42761478 0.78619231 1.39654444]
0.9809867099531238 [0.40225841 0.71326384 1.31399089]
1.1024099210157456 [0.46918946 0.91479603 1.57767368]
1.0650250670624901 [0.4458092  0.86315034 1.48338553]
1.1207492667079197 [0.48468746 0.95120223 1.64707625]
1.130994243664058 [0.49043739 0.95828698 1.68324932]
1.1514531851750545 [0.49824042 0.98192637 1.73739969]
1.1667664809954692 [0.51267979 1.